# Spoken Language Processing 2023-24

# Lab3 - Dialogue Systems

_Bruno Martins_


This lab assignment will introduce tools and concepts related to the development of dialogue systems, exemplifying also the use of automatic speech recognition and text-to-speech models in this particular context.

Students will be tasked with the development of a simple (spoken/conversational) question answering system, reusing different models associated to the HuggingFace Transformers library:

* Speech recognition models (e.g., OpenAI Whisper).
* Large language models for natural language understanding and generation (e.g., [LaMini-GPT2](https://huggingface.co/MBZUAI/LaMini-GPT-124M) models).
* Text-to-speech models (e.g., SpeechT5).

The first parts of this notebook will guide students in the use of the tools, while the last part presents the main problem that is to be tackled. Note that the first parts also feature intermediate tasks, which students are required to solve.

To complete the project, student groups must deliver in Fenix an updated version of this notebook, featuring the proposed solutions to each task, together with a small PDF report (2 pages) outlining the methods that were developed (you can use the [following Overleaf template](https://www.overleaf.com/latex/templates/interspeech-2023-paper-kit/kzcdqdmkqvbr) for the report). The report can contain a section for each of the parts in the notebook, and the set of two documents should be uploaded in Fenix through a .zip file named after the number of the group.

Students are encouraged to modify examples, incorporate any other techniques, and in general explore any approach that may permit improving the results. Assessment will be based on task completion, creativity in the proposed solutions, and overall accuracy over a benchmark dataset.

### Group identification

Initialize the variable `group_id` with the number that Fenix assigned to your group and `student1_name`, `student1_id`, `student2_name` and `student2_id` with your names and student numbers.

In [1]:
group_id = 4
student1_name = "Margarida Pereira"
student2_name = "Piotr Migdałek"
student1_id = 111111
student2_id = 108765

print(f"Group number: {group_id}")
print(f"Student 1: {student1_name} ({student1_id})")
print(f"Student 2: {student2_name} ({student2_id})")

Group number: 4
Student 1: Margarida Pereira (111111)
Student 2: Piotr Migdałek (108765)


In [2]:
assert isinstance(group_id, int) and isinstance(student1_id, int) and isinstance(student2_id, int)
assert isinstance(student1_name, str) and isinstance(student2_name, str)
assert (group_id > 0) and (group_id < 40)
assert (student1_id > 60000) and (student1_id < 120000) and (student2_id > 60000) and (student2_id < 120000)

# Install and import Python packages

NumPy is a Python library that provides functions to process multidimensional arrays. The NumPy documentation is available [here](https://numpy.org/doc/1.24/).

[Librosa](https://librosa.org/) is a Python package for analyzing and processing audio signals. It provides a wide range of tools for tasks such as loading and manipulating audio files, extracting features from audio signals, and visualizing and playing back audio data.

IPython display is a module in the IPython interactive computing environment that provides a set of functions for displaying various types of media in the Jupyter notebook or other IPython-compatible environments. For example, you can use the display() function to display an object in a notebook cell (for example an audio object).

Matplotlib is a popular Python library that allows users to create a wide range of visualizations using a simple and intuitive syntax.

Huggingface transformers provides APIs and tools to easily download and train state-of-the-art pretrained models based on the Transformer architecture. The documentation is available [here](https://huggingface.co/docs/transformers/index) and, for more details, look at the official [HuggingFace course](https://huggingface.co/course/chapter1/1).

The associated HuggingFace libraries named [datasets](https://huggingface.co/docs/datasets/index) and [evaluate](https://huggingface.co/docs/evaluate/index) respectivly suport the direct access to many well-known datasets and common evaluation metrics used in NLP and speech research.

In [3]:
!pip3 install sentencepiece
!pip3 install xformers
!pip3 install transformers
!pip3 install datasets
!pip3 install evaluate
!pip3 install jiwer
!pip3 install librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 19.9 MB/s eta 0:00:0000:0100:01
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.4 MB/s eta 0:00:00a 0:00:01
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 58.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/

In [4]:
import transformers
import datasets
import evaluate
import numpy as np
import librosa
import librosa.display
from IPython.display import Audio
from matplotlib import pyplot as plt

/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Using OpenAI Whisper

Whisper is a cutting-edge model for for Automatic Speech Recognition (ASR), developed by OpenAI using a massive dataset of 680,000 hours of multilingual and multitask supervised data collected from the internet, and made available through the HuggingFace Transformers library.

The following example illustrates the use of the Whisper model to transcribe a small audio sample taken from the LibriSpeech dataset (which is available through the HuggingFace datasets library).

More detailed information about Whisper, including information on how to fine-tune the model with task-specific data, is available on a [tutorial in the HuggingFace blog](https://huggingface.co/blog/fine-tune-whisper).

In [5]:
import torch
import librosa
from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

processor = AutoProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

audio = ds[0]["audio"]["array"]
audio = librosa.resample(audio, orig_sr=16000, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)

inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

display(Audio(audio, rate=16000)) # You are able to hear the audio inputs

generated_ids = model.generate(inputs=input_features)
transcription = processor.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]

print(transcription)

/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for hf-internal-testing/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hf-internal-testing/librispeech_asr_dummy
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating validation split: 73 examples [00:00, 9042.65 examples/s]
/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:473: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.


Automatic Speech Recognition (ASR) models are frequently evaluated through the Word Error Rate ([WER](https://huggingface.co/spaces/evaluate-metric/wer)).

The WER is derived from the Levenshtein distance, working at the word level and aligning the recognized word sequence with the reference (spoken) word sequence using dynamic string alignment. The metric can then be computed as:

WER = (S + D + I) / N = (S + D + I) / (S + D + C),

where S is the number of substitutions, D is the number of deletions, I is the number of insertions, C is the number of correct words, and N is the number of words in the reference (N=S+D+C). The WER value indicates the average number of errors per reference word. The lower the value, the better the performance of the ASR system, with a WER of 0 being a perfect score.

The example below illustrates the computation of the WER for two paired examples of a generated sentence versus a reference sentence. The score produced as output is the average value accross the two examples.

In [6]:
from evaluate import load

wer = load("wer")
predictions = ["this is the prediction", "there is an other sample"]
references = ["this is the reference", "there is another one"]
wer_score = wer.compute(predictions=predictions, references=references)

print(wer_score)

0.5


## Intermediate tasks:

* Collect two small audio samples with your own voice, together with a transcription of the spoken messages. The following [example shows how to record audio from your microphone within a Python notebook running on Google Colab](https://colab.research.google.com/gist/ricardodeazambuja/03ac98c31e87caf284f7b06286ebf7fd/microphone-to-numpy-array-from-your-browser-in-colab.ipynb#scrollTo=H4rxNhsEpr-c), but you can use any other method to collect the audio samples.
* Use the Whisper speech recognition model to transcribe the two spoken messages that were collected.
* Use the transcriptions to compute the word error rate.
* Experiment with the use of different recognition models (e.g., larger Whisper models), and see if the error rate changes.

In [21]:
peasants, sr = librosa.load("peasants.wav", sr=44100)
bread, sr = librosa.load("bread.wav", sr=44100)

In [26]:
transcript_peasants = "The Peasants is a Polish novel from 19th century."
transcript_bread = "A sturdy bread on a read table."

In [23]:
peasants = librosa.resample(peasants.astype(np.float32), orig_sr=44100, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)

inputs = processor(peasants, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

display(Audio(peasants, rate=16000)) # You are able to hear the audio inputs

generated_ids = model.generate(inputs=input_features)
transcription = processor.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]

print(transcription)

 The peasants is a Polish novel from 19th century.


In [18]:
wer_score = wer.compute(predictions=[transcription], references=[transcript_peasants])
print(wer_score)

0.1111111111111111


In [19]:
wer_score = wer.compute(predictions=[transcription.lower()], references=[transcript_peasants.lower()])
print(wer_score)

0.0


In [29]:
bread = librosa.resample(bread.astype(np.float32), orig_sr=44100, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)

inputs = processor(bread, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

display(Audio(bread, rate=16000)) # You are able to hear the audio inputs

generated_ids = model.generate(inputs=input_features)
transcription = processor.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]

print(transcription)

/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:473: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


 A sturdy bread on a red table.


In [30]:
wer_score = wer.compute(predictions=[transcription], references=[transcript_bread])
print(wer_score)

0.14285714285714285


In [32]:
print(transcription)
print(transcript_bread)

 A sturdy bread on a red table.
A sturdy bread on a read table.


We see that using simple sentences tiny Whisper does a great jobs (almost) perfectly transcribing given files. But let's see how he deals with some poetry.

In [33]:
transcript_blake = "Tyger Tyger, burning bright, In the forests of the night; What immortal hand or eye, Could frame thy fearful symmetry?"
blake, sr = librosa.load("blake.wav", sr=44100)

In [34]:
blake = librosa.resample(blake.astype(np.float32), orig_sr=44100, target_sr=16000) # Resample audio to 16kHz (not needed in the case of this dataset)

inputs = processor(blake, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

display(Audio(blake, rate=16000)) # You are able to hear the audio inputs

generated_ids = model.generate(inputs=input_features)
transcription = processor.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]

print(transcription)

/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:473: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


 Tiger, Tiger, Burning, Bright in the forest of the night, what immortal Heather I could friend of fearful symmetry.


In [35]:
wer_score = wer.compute(predictions=[transcription], references=[transcript_blake])
print(wer_score)

0.75


In [36]:
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")

/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
inputs = processor(blake, sampling_rate=16000, return_tensors="pt")
input_features = inputs.input_features

display(Audio(blake, rate=16000)) # You are able to hear the audio inputs

generated_ids = model.generate(inputs=input_features)
transcription = processor.batch_decode(generated_ids, max_length=250, skip_special_tokens=True)[0]

print(transcription)

/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:473: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Tiger, tiger, burning bright, In the forest of the night, What immortal hand or eye Could frame their fearful symmetry?


In [38]:
wer_score = wer.compute(predictions=[transcription], references=[transcript_blake])
print(wer_score)

0.3


In [40]:
!pip install huggingsound

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    Uninstalling torch-2.3.0:
      Successfully uninstalled torch-2.3.0
  Attempting uninstall: rapidfuzz
    Found existing installation: rapidfuzz 3.9.1
    Uninstalling rapidfuzz-3.9.1:
      Successfully uninstalled rapidfuzz-3.9.1
  Attempting uninstall: jiwer
    Found existing installation: jiwer 3.0.4
    Uninstalling jiwer-3.0.4:
      Successfully uninstalled jiwer-3.0.4
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.2.post1
    Uninstalling librosa-0.10.2.post1:
      Successfully uninstalled librosa-0.10.2.post1
ERROR: pip's dependen

In [41]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english")
audio_paths = ["blake.wav"]

transcription = model.transcribe(audio_paths)

05/23/2024 20:02:42 - INFO - huggingsound.speech_recognition.model - Loading model...


/home/pmigdalek/spoken-language-processing-23-24-IST/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeq

In [48]:
print(transcription[0]['transcription'])
wer_score = wer.compute(predictions=[transcription[0]['transcription']], references=[transcript_blake])
print(wer_score)

tiger tager burning bright in the forest of the night what immortal hander i could frind the fearful symmetry
0.7


Whisper Large improved from from its Tiny version from 0.75 to 0.3 WER, yielding great transcription (missing only the old english influences and potery signs). wav2vec2-large transcription didn't include any punctuation, thus finally yielding WER of 0.7

# Using LLMs for conditional language generation

OpenAI GPT-2 is a language model based on the Transformer decoder architecture, trained with large scale data collected from the Web using a simple objective: predict the next word, given all of the previous words within some text. The diversity of the dataset causes this simple goal to contain naturally occurring demonstrations of many tasks across diverse domains. Thus, GPT-2 can be used to address problems like question answering, modeling the task as language generation conditioned in the question (plus other relevant additional context).

The following example illustrates the use of the GPT-2 through the Huggingface Transformers library. In this case, instead of using the model directly, we are using the model through the pipeline API, which facilitates the adaptation to the case of other LLMs. The pipeline() function can be used to connect a model with its necessary preprocessing and postprocessing steps, allowing us to directly input any text and get an intelligible answer.

In [ ]:
from transformers import pipeline, set_seed

set_seed(42) # make results deterministic

generator = pipeline(model='gpt2')
generator("Who is the president of the United States? The answer is", max_length=15, num_return_sequences=1)

## Intermediate tasks:

* Adapt the example showing how to use GPT-2 to do question answering over the [TriviaQA dataset](http://nlp.cs.washington.edu/triviaqa/) (you can use a [version](https://huggingface.co/datasets/lucadiliello/triviaqa) of this dataset from a previous shared task, which is available from HuggingFace datasets).
* Evaluate the results obtained with different models (e.g., relatively small models trained to follow instructions, for instance from the [LaMini-LM](https://huggingface.co/MBZUAI/LaMini-GPT-124M) family, or quantized versions of larger models such as [Meta Llama 3](https://huggingface.co/meta-llama/Meta-Llama-3-8B)) and/or different usage strategies (e.g., consider different prompting strategies, parameter efficient fine-tuning, etc.).
* Compute the error over the first 1000 examples from the validation split from the TriviaQA dataset, using the [BLEU metric](https://huggingface.co/spaces/evaluate-metric/bleu) for comparing the generated answers against the ground truth.


In [ ]:
# Add your solutions to the exercises

# Using SpeechT5 for converting text-to-speech

Motivated by the success of T5 (Text-To-Text Transfer Transformer) in different natural language processing tasks, the unified-modal SpeechT5 framework explores encoder-decoder pre-training for self-supervised speech/text representation learning.

The model is again conveniently available through the HuggingFace Transformers library. The following example illustrates the use of the SpeechT5 model for generating a spectrogram from a textual input, together with a neural vocoder model for producing a speech signal.

More detailed information about SpeechT5 is available on a [tutorial on the HuggingFace blog](https://huggingface.co/blog/speecht5).

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan, set_seed
from IPython.display import Audio
from datasets import load_dataset
import soundfile as sf
import librosa
import torch

set_seed(42) # make results deterministic

model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

inputs = processor(text="Hello, my dog is cute.", return_tensors="pt")
speaker_embeddings = torch.zeros((1, 512))

# When using SpeechT5 for TTS, you should use "xvector speaker embeddings"
# to customize the output to a particular speaker’s voice characteristics
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[42]["xvector"]).unsqueeze(0)

spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)
with torch.no_grad(): speech = vocoder(spectrogram)

display(Audio(speech.numpy(), rate=16000)) # You can hear the audio inputs

# You can plot the generated spectrogram
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(spectrogram.T)
plt.show()

librosa.display.waveshow(speech.numpy(), sr=16000) # You can plot the generated waveform

sf.write("tts_example.wav", speech.numpy(), samplerate=16000) # You can save the audio to a .wav file

## Intermediate tasks:

* Connect the results from your answer to the previous intermediate task (i.e., conditioned language generation) to the SpeechT5 text-to-speech model, so as to produce speech outputs from the text generated by the model.
* Produce speech-based answers for the first 5 questions in the validation split from the TriviaQA dataset.
* Connect also the results from your answer to the first intermediate task (i.e., automated speech recognition) to the SpeechT5 model and the LLM, so as to take spoken questions as input and produce a speech output.
* Take the audio samples from 10 TriviaQA questions (as available in connection to the [SLUE-SQA-5 dataset](https://huggingface.co/datasets/asapp/slue-phase-2), in Huggingface datasets), and evaluate the answers generated for the spoken questions using the BLEU metric.
* Collect small audio samples, with your own voice, for the first 2 questions in the validation split from the TriviaQA dataset, and produce speech-based answers for these two questions.


In [ ]:
# Add your solutions to the exercises

# Main problem

Students are tasked with joining together the speech recognition, language understanding and generation, and text-to-speech models, in order to build a conversational spoken question answering approach.

* The method should take as input speech utterances with questions.
* The language understanding and generation component should use as input a transcription for the current speech utterance, and optionally also transcriptions from previous speech utterances (i.e., the conversation context).
* The language understanding and generation component can explore different strategies for improving answer quality:
  * Use of large LLMs trained to follow instructions, e.g. with reinforcement learning from human feedback.
  * Prompting the language model with retrieved in-context examples.
  * Using parameter-efficient fine-ting with existing conversational question answering datasets (e.g., [the CoQA dataset](https://stanfordnlp.github.io/coqa/), which is [also available](https://huggingface.co/datasets/stanfordnlp/coqa) from HuggingFace datasets).
  * ...
* The text-to-speech component takes as input the results from language generation, and produces a speech output.
* Both the automated speech recognition and the text-to-speech components can explore different approaches, although students should attempt to justify their choices (e.g., if changing the automated speech recognition component, show that it achieves a lower WER).
* Collect small audio samples, with your own voice, for the questions in the first instance in the CoQA validation split, and show the results produced by your method for these questions.

In [ ]:
# Add your solutions to the exercises